In [ ]:
# load package
library(GeneNet)
packageVersion('GeneNet')
library(dplyr)
packageVersion('dplyr')

In [ ]:
# set directory
project.dir = '...'
data.dir = '...'
results.dir = '...'

In [ ]:
# load QC ed metabolomics data of data cohort
data.met <- read.csv(file.path(data.dir, 'Metabolites_QC.csv'))
dim(data.met)
length(unique(data.met$Subject_Id)) # 711 subjects
head(data.met)

In [ ]:
# estimate of partial correlations
estimated.pcor <- ggm.estimate.pcor(data.met[,-c(1)]) #static and shrinkage
dim(estimated.pcor)
head(estimated.pcor)

In [ ]:
met.node.id <- data.frame(colnames(estimated.pcor),c(1:810))
colnames(met.node.id) <- c('metabolite','node.id')
dim(met.node.id)
head(met.node.id)

In [ ]:
# merge to have metabolite names
met.info <- read.csv(file.path(data.dir, 'Metabolites_ID_final.csv'))
met.node.id <- met.node.id %>% left_join(met.info[,c('metabolite','CHEMICAL_NAME')], by = 'metabolite')
dim(met.node.id)
head(met.node.id)

In [ ]:
# p-values, q-values and posterior probabilities for each potential edge
node.results <- network.test.edges(estimated.pcor)
node.results

In [ ]:
node.results.w.med.id <- merge(node.results,met.node.id, by.x = 'node1', by.y = 'node.id')
dim(node.results.w.med.id)
head(node.results.w.med.id)

In [ ]:
node.results <- node.results[,c(1:5)] # select 1st five columns
node.results 

In [ ]:
colnames(node.results.w.med.id)[colnames(node.results.w.med.id) == 'CHEMICAL_NAME'] <- 'metabolite.1'
head(node.results.w.med.id)

In [ ]:
node.results.w.med.id <- merge(node.results.w.med.id,met.node.id, by.x = 'node2', by.y = 'node.id')
dim(node.results.w.med.id)
head(node.results.w.med.id)

In [ ]:
colnames(node.results.w.med.id)[colnames(node.results.w.med.id) == 'CHEMICAL_NAME'] <- 'metabolite.2'
head(node.results.w.med.id)

In [ ]:
node.results.w.med.id <- node.results.w.med.id %>% select(metabolite.1,metabolite.2,pcor,pval,qval)
dim(node.results.w.med.id)
head(node.results.w.med.id)

In [ ]:
# filter only significant edges based on FDR cut off of 0.05
sig.corr.05 <- node.results.w.med.id %>% filter(qval < 0.05)
sig.corr.05

In [ ]:
dim(sig.corr.05) # number of edges
length(unique(c(sig.corr.05$`metabolite.1`,sig.corr.05$`metabolite.2`))) # number of nodes:

In [ ]:
# load significant metabolites from linear model
sig.mets <- read.csv(file.path(results.dir, 'nominally_significant_metabolites_adrenal_suppression.csv'))
dim(sig.mets)
head(sig.mets)

In [ ]:
sig.mets <- sig.mets %>% select(metabolite,CHEMICAL_NAME,SUPER_PATHWAY,SUB_PATHWAY)
head(sig.mets)

In [ ]:
# filter only significant nodes
sig.mets.corr <- filter(sig.corr.05, metabolite.1 %in% sig.mets$CHEMICAL_NAME & metabolite.2 %in% sig.mets$CHEMICAL_NAME)
sig.mets.corr

In [ ]:
length(unique(c(sig.mets.corr$`metabolite.1`,sig.mets.corr$`metabolite.2`)))
dim(sig.mets.corr)

In [ ]:
summary(sig.mets.corr$pcor)

In [ ]:
write.csv(sig.mets.corr, file.path(results.dir, 'partial_correlation_nominally_sig_mets_from_linear_models.csv'), row.names = FALSE)